In [1]:
import requests, re, wikipediaapi, pygraphviz, pickle
import pandas as pd, networkx as nx
import numpy as np
from datetime import datetime as dt
#from networkx.drawing.nx_agraph import graphviz_layout
from bs4 import BeautifulSoup
user_agent = "MSDS452/Assignment1 (https://example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent)

# import bokeh
# from bokeh.io import push_notebook, show, output_notebook, save, output_file
# import bokeh.plotting as bp
# from bokeh.plotting import figure, save, output_file, show 
# from bokeh.models import (ColumnDataSource, LabelSet, Label, BoxSelectTool, Circle, EdgesAndLinkedNodes, HoverTool,MultiLine, NodesAndLinkedEdges, Plot, Range1d, TapTool,)
# from holoviews.element.graphs import layout_nodes

# output_notebook()
# import holoviews as hv
# from holoviews import dim, opts
# hv.extension('bokeh', 'matplotlib')
# from holoviews.operation import  gridmatrix
# from holoviews.operation.datashader import datashade, bundle_graph
# from holoviews import Graph, Nodes
# from holoviews.plotting.bokeh import GraphPlot, LabelsPlot
# import hvplot.networkx as hvnx
# import hvplot.pandas

# import functions from utils
from utils.scrape_functions import scrape_ballotpedia_tables, get_ballotpedia_table, check_wikipedia_page, scrape_wikipedia_tables, href_wiki, scrape_wikipedia_table, find_state_name
#, html_to_text_with_hyperlinks, extract_urls
#, node_sizes_scaling, hv_plot_graph, graph_attributes

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.simplefilter('ignore')

## Initialize common variables used throughout code

In [2]:
state_codes = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT',
    'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY','District of Columbia':'DC','Guam':'GU',
    'Samoa':'AS','Mariana Islands':'MP','Virgin Islands':'VI'
}

# Status values that means the candidate will not be on the ballot
not_running = ['Withdrew Primary', 'Withdrew General', 'Lost Primary', 'Lost (Write-in) Primary','Withdrew (Write-in) Primary',
               'Disqualified Primary', 'Withdrew Round 1','Disqualified General','Lost Primary Runoff','Lost (unofficially withdrew) Primary',
               'Lost (Write-in) Primary','Withdrew Round 1','Withdrew (Write-in) General','Withdrew (unofficially withdrew) Primary',
               'Lost Convention']


# US Senate Candidates

## Scrape Ballotpedia tables of candidates by state

In [3]:
senate_url = 'https://ballotpedia.org/United_States_Senate_elections,_2024'

# Get all tables on the Ballotpedia page
senate_tables = scrape_ballotpedia_tables(senate_url)

# Initialize an empty list to store tables as DataFrames
dfs_list = []

# Iterate over all of the tables and parse candidate tables to dataframe
for i, table in enumerate(senate_tables):
    # Use table's id tag to identify tables we want to scrape
    table_id = table.get('id') 
    
    if table_id != None:    # all of the candidates table have an id tag
        dfv = get_ballotpedia_table(senate_tables, i)
        state_name = find_state_name(dfv.iloc[0]['office'])
        dfv['State'] = state_codes[state_name]
        dfs_list.append(dfv)
        print(state_name,"done")

# Concatenate all DataFrames in the list dfs
dfs = pd.concat(dfs_list, ignore_index=True)

print(dfs.shape) 

Arizona done
California done
Connecticut done
Delaware done
Florida done
Hawaii done
Indiana done
Maine done
Maryland done
Massachusetts done
Michigan done
Minnesota done
Mississippi done
Missouri done
Montana done
Nebraska done
Nevada done
New Jersey done
New Mexico done
New York done
North Dakota done
Ohio done
Pennsylvania done
Rhode Island done
Tennessee done
Texas done
Utah done
Vermont done
Virginia done
Washington done
West Virginia done
Wisconsin done
Wyoming done
(500, 5)


## Clean Senate Candidates DataFrame

In [4]:
# Clean candidate column to remove "Incumbent" and add as a variable in new column
# Check for Wikipedia page using cleaned name
dfs0=dfs.copy()
ccand=[]
incumbency=[]
gender=["" for i in range(len(dfs0))]
for i in range(len(dfs0)):
    cand=dfs0.iloc[i]['candidate']
    if "Incumbent" in cand:
        incumbency.append("Incumbent")
        ccand.append(cand.replace(" Incumbent",""))
    else:
        incumbency.append(None)
        ccand.append(cand)
dfs0["candidate_mod"]=ccand
dfs0["incumbency"]=incumbency
dfs0["gender"]=gender
dfs0['Wiki page'] = dfs0['candidate_mod'].apply(check_wikipedia_page)
dfs0=dfs0[['candidate_mod', 'incumbency', 'gender', 'party', 'office', 'status', 'State', 'Wiki page']]
dfs0.rename(columns={'candidate_mod':'candidate'}, inplace=True)
dfs = dfs0

In [5]:
# Scrape the Wikipedia table for candidates to use to validate candidate Wikipedia pages
senate_wiki_url = 'https://en.wikipedia.org/wiki/2024_United_States_Senate_elections'
senate_wiki_tables = scrape_wikipedia_tables(senate_wiki_url)

senate_wiki_links = []

# There is a single table will all Senate candidates
table = senate_wiki_tables[11]
for row in table.find_all('tr'):
    cols = row.find_all('td')
    if len(cols)==0:
        continue
    col = cols[-1]   #Candidates is last column in table
    links = col.find_all(href=href_wiki)
    
    for link in links:
        senate_wiki_links.append(link.get('href'))


In [6]:
wp = [w.split("/", 2)[2].replace("_", " ") if len(w.split("/", 2)) >= 3 else "" for w in senate_wiki_links]
wpc = [w[:w.rfind(" (")] if " (" in w else w for w in wp]
dfs['Wiki page (validated)'] = None

# Compare scraped and cleaned scraped wiki pages from table to candidate name in dataframe
for i in range(len(dfs)):
    if dfs.at[i, "candidate"] in wp:
        dfs.loc[i, 'Wiki page (validated)'] = dfs.at[i, "candidate"]
    else:
        if dfs.at[i, "candidate"] in wpc:
            dfs.loc[i, 'Wiki page (validated)'] = wp[wpc.index(dfs.at[i, "candidate"])]

### Manual attempts to find missing candidate's Wikipedia pages

#### Print candidates with a status indicating they will be on the ballot (i.e. not withdrawn or lost primary) and do not have a Wikipedia page

In [7]:
# # List candidates that have are missing Wiki page and Wiki page (validated)
# sp=list(zip(dfs['candidate'],dfs['office'],zip(dfs['Wiki page'],dfs['Wiki page (validated)'])))
# count_total=0    #count number of candidates that are missing wiki pages
# count_running=0  #count number of candidates that have a status indicating they will be on the ballot
# for s in sp:
#     if s[2]==(None,None):
#         # Updated to only include candidates that will be on the ballot
#         if not (dfs.loc[dfs['candidate']==s[0], 'status'].isin(not_running).any()):
#             print(s)
#             count_running+=1
#         count_total+=1

# print("total", count_total)
# print("running", count_running)

In [8]:
# # List candidates that have are have Wiki page but are missing Wiki page (validated)
# sp=list(zip(dfs['candidate'],zip(dfs['Wiki page'],dfs['Wiki page (validated)'])))
# count_total=0    #count number of candidates that have a wiki page but not validated
# count_running=0  #count number of candidates that have a status indicating they will be on the ballot
# for s in sp:
#     if s[1][0] != None and s[1][1]==None:
#         # Updated to only include candidates that will be on the ballot
#         if not (dfs.loc[dfs['candidate']==s[0], 'status'].isin(not_running).any()):
#             print(s)
#             count_running+=1
#         count_total+=1

# print("total", count_total)
# print("running", count_running)

#### Add Wikipedia pages that were manually found to the dataframe

In [14]:
# Found following wiki links that were missing
missing_senate_wikis = {'Robert Hyde': 'https://en.wikipedia.org/wiki/Robert_F._Hyde',
'Alan Grayson': 'https://en.wikipedia.org/wiki/Alan_Grayson',
'Randy Toler': 'https://en.wikipedia.org/wiki/Randy_Toler',
'Shiva Ayyadurai': 'https://en.wikipedia.org/wiki/Shiva_Ayyadurai',
'Mike Schaefer': 'https://en.wikipedia.org/wiki/Mike_Schaefer',
'Andrew Kim': 'https://en.wikipedia.org/wiki/Andy_Kim_(politician)',
'Alexander Mooney': 'https://en.wikipedia.org/wiki/Alex_Mooney',
'Mazie K. Hirono': 'https://en.wikipedia.org/wiki/Mazie_Hirono',
'Jeff Gunter': 'https://en.wikipedia.org/wiki/Jeffrey_Ross_Gunter',
'Brad R. Wilson': 'https://en.wikipedia.org/wiki/Brad_Wilson_(politician)',
'Cara Castronuova': 'https://en.wikipedia.org/wiki/Cara_Castronuova',
'Brad Johnson': 'https://en.wikipedia.org/wiki/Brad_Johnson_(Montana_politician)',
'David Trone': 'https://en.wikipedia.org/wiki/David_Trone',
'Don Blankenship': 'https://en.wikipedia.org/wiki/Don_Blankenship',
'Garn Mabey': 'https://en.wikipedia.org/wiki/Garn_Mabey',
'Jim Marchant': 'https://en.wikipedia.org/wiki/Jim_Marchant',
'Marc Carmichael': 'https://en.wikipedia.org/wiki/Marc_Carmichael',
'Nasser Beydoun': 'https://en.wikipedia.org/wiki/Nasser_Beydoun',
'Peter Meijer': 'https://en.wikipedia.org/wiki/Peter_Meijer',
'Robin Ficker': 'https://en.wikipedia.org/wiki/Robin_Ficker'
}

for k,v in missing_senate_wikis.items():
    i = dfs.index[dfs['candidate'] == k][0]
    
    val_wiki = v.split("/", 4)[4].replace("_", " ")
    
    dfs.loc[i, 'Wiki page (validated)'] = val_wiki


### Address candidate's listed multiple times (same names)

In [15]:
# There are some candidates listed multiple times
grouped_cand = dfs.groupby('candidate')

# Filter groups with more than one row
filtered_groups = grouped_cand.filter(lambda x: len(x) > 1)

dup_names = filtered_groups['candidate'].unique()
update_count = {}
def update_name(name):
    if name not in dup_names:
        return name
        
    if name in update_count:
        update_count[name] += 1
    else:
        update_count[name] = 1

    count = update_count[name]
    return f"{name} ({count})"

dfs['candidate'] = dfs['candidate'].apply(update_name)
dfs['incumbency'] = dfs['incumbency'].fillna('Challenger')


### Save the dataframe to a pickle

In [16]:
date = dt.now().strftime('%d%b%Y')
dfs.to_pickle(f'../data/US_Senate_2024_df_updated_{date}.pickle')

# US House Candidates

In [3]:
house_url = 'https://ballotpedia.org/United_States_House_of_Representatives_elections,_2024'

house_tables = scrape_ballotpedia_tables(house_url)

# Initialize an empty list to store DataFrames
dfh_list = []

for i, table in enumerate(house_tables):
    # Use table's id tag to identify tables we want to scrape
    table_id = table.get('id')
    
    if table_id != None:    # all of the candidates table have an id tag
        dfv = get_ballotpedia_table(house_tables, i)
        #dfv['Wiki page'] = dfv['candidate'].apply(check_wikipedia_page)
        state_name = find_state_name(dfv.iloc[0]['office'])
        dfv['State'] = state_codes[state_name]
        dfh_list.append(dfv)
        print(state_name,"done")

# Concatenate all DataFrames in the list dfs
dfh = pd.concat(dfh_list, ignore_index=True)
print(dfh.shape)

Alabama done
Alaska done
Samoa done
Arizona done
Arkansas done
California done
Colorado done
Connecticut done
Delaware done
District of Columbia done
Florida done
Georgia done
Guam done
Hawaii done
Idaho done
Illinois done
Indiana done
Iowa done
Kansas done
Kentucky done
Louisiana done
Maine done
Maryland done
Massachusetts done
Michigan done
Minnesota done
Mississippi done
Missouri done
Montana done
Nebraska done
Nevada done
New Hampshire done
New Jersey done
New Mexico done
New York done
North Carolina done
North Dakota done
Mariana Islands done
Ohio done
Oklahoma done
Oregon done
Pennsylvania done
Rhode Island done
South Carolina done
South Dakota done
Tennessee done
Texas done
Utah done
Vermont done
Virgin Islands done
Virginia done
Washington done
West Virginia done
Wisconsin done
Wyoming done
(2722, 5)


##  Clean House Candidates Dataframe

In [5]:
# Clean candidate column to remove "Incumbent" and add as a variable in new column
# Check for Wikipedia page using cleaned name
dfh0=dfh.copy()
ccand=[]
incumbency=[]
gender=["" for i in range(len(dfh0))]
for i in range(len(dfh0)):
    cand=dfh0.iloc[i]['candidate']
    if "Incumbent" in cand:
        incumbency.append("Incumbent")
        ccand.append(cand.replace(" Incumbent",""))
    else:
        incumbency.append(None)
        ccand.append(cand)

dfh0["candidate_mod"]=ccand
dfh0["incumbency"]=incumbency
dfh0["gender"]=gender
dfh0['Wiki page'] = dfh0['candidate_mod'].apply(check_wikipedia_page)
dfh0=dfh0[['candidate_mod', 'incumbency', 'gender', 'party', 'office', 'status', 'State', 'Wiki page']]
dfh0.rename(columns={'candidate_mod':'candidate'}, inplace=True)
dfh = dfh0

In [6]:
# Scrape the wikipedia tables for House candidates to use to find candidates wiki pages
house_wiki_url = 'https://en.wikipedia.org/wiki/2024_United_States_House_of_Representatives_elections'
house_wiki_tables = scrape_wikipedia_tables(house_wiki_url)

house_wiki_links = []

# Alabama is table number 8 and districts are 58
for i in range(8,59):
    table = house_wiki_tables[i]
    for row in table.find_all('tr'):
        cols = row.find_all('td')
        if len(cols)==0:
            continue
        col = cols[-1]   #Candidates is last column in table
        links = col.find_all(href=href_wiki)
        
        for link in links:
            house_wiki_links.append(link.get('href'))


In [7]:
wp = [w.split("/", 2)[2].replace("_", " ") if len(w.split("/", 2)) >= 3 else "" for w in house_wiki_links]
wpc = [w[:w.rfind(" (")] if " (" in w else w for w in wp]
dfh['Wiki page (validated)'] = None

# Compare scraped and cleaned scraped wiki pages from table to candidate name in dataframe
for i in range(len(dfh)):
    if dfh.at[i, "candidate"] in wp:
        dfh.loc[i, 'Wiki page (validated)'] = dfh.at[i, "candidate"]
    else:
        if dfh.at[i, "candidate"] in wpc:
            dfh.loc[i, 'Wiki page (validated)'] = wp[wpc.index(dfh.at[i, "candidate"])]

### Manual attempts to find missing candidate's Wikipedia pages

#### Print candidates with a status indicating they will be on the ballot (i.e. not withdrawn or lost primary) and do not have a Wikipedia page

In [8]:
# # List candidates that have are missing Wiki page and Wiki page (validated)
# hp=list(zip(dfh['candidate'],dfh['office'],zip(dfh['Wiki page'],dfh['Wiki page (validated)'])))
# count_total=0    #count number of candidates that are missing wiki pages
# count_running=0  #count number of candidates that have a status indicating they will be on the ballot
# for h in hp:
#     if h[2]==(None,None):
#         # Updated to only include candidates that will be on the ballot
#         if not (dfh.loc[dfh['candidate']==h[0], 'status'].isin(not_running).any()):
#             print(h)
#             count_running+=1
#         count_total+=1

# print("total", count_total)
# print("running", count_running)

In [9]:
# # List candidates that have are have Wiki page but are missing Wiki page (validated)
# count_total=0    #count number of candidates that have a wiki page but not validated
# count_running=0  #count number of candidates that have a status indicating they will be on the ballot
# for h in hp:
#     if h[2][0] != None and h[2][1]==None:
#         # Updated to only include candidates that will be on the ballot
#         if not (dfh.loc[dfh['candidate']==h[0], 'status'].isin(not_running).any()):
#             print(h)
#             count_running+=1
#         count_total+=1

# print("total", count_total)
# print("running", count_running)

In [10]:
# Found following wiki links that were missing for House
# Note that full check has not been completed
missing_house_wikis = {'Kevin Lincoln II': 'https://en.wikipedia.org/wiki/Kevin_Lincoln_(politician)',
 'Luz Maria Rivas': 'https://en.wikipedia.org/wiki/Luz_Rivas',
 'Jared Evan Moskowitz': 'https://en.wikipedia.org/wiki/Jared_Moskowitz',
 'Geoff M. Young': 'https://en.wikipedia.org/wiki/Geoff_Young',
 'John Olszewski Jr.': 'https://en.wikipedia.org/wiki/Johnny_Olszewski_(politician)',
 'John B. Driscoll': 'https://en.wikipedia.org/wiki/John_Driscoll_(Montana_politician)',
 'Raquel Terán': 'https://en.wikipedia.org/wiki/Raquel_Ter%C3%A1n',
 'Raúl Grijalva': 'https://en.wikipedia.org/wiki/Ra%C3%BAl_Grijalva',
 'David G. Valadao': 'https://en.wikipedia.org/wiki/David_Valadao',
 'George Whitesides': 'https://en.wikipedia.org/wiki/George_T._Whitesides',
 'Linda Sánchez': 'https://en.wikipedia.org/wiki/Linda_S%C3%A1nchez',
 'Nanette Barragán': 'https://en.wikipedia.org/wiki/Nanette_Barrag%C3%A1n',
 'Michael Lynch': 'https://en.wikipedia.org/wiki/Mike_Lynch_(Colorado_politician)',
 'John Larson': 'https://en.wikipedia.org/wiki/John_B._Larson',
 'Rosa L. DeLauro': 'https://en.wikipedia.org/wiki/Rosa_DeLauro',
 'John Quiñones': 'https://en.wikipedia.org/wiki/John_Qui%C3%B1ones_(politician)',
 'Maxwell Alejandro Frost': 'https://en.wikipedia.org/wiki/Maxwell_Frost',
 'Gus M. Bilirakis': 'https://en.wikipedia.org/wiki/Gus_Bilirakis',
 'Frederica S. Wilson': 'https://en.wikipedia.org/wiki/Frederica_Wilson',
 'Mario Diaz-Balart': 'https://en.wikipedia.org/wiki/Mario_D%C3%ADaz-Balart',
 'Maria Elvira Salazar': 'https://en.wikipedia.org/wiki/Mar%C3%ADa_Elvira_Salazar',
 'Carlos Gimenez': 'https://en.wikipedia.org/wiki/Carlos_A._Gim%C3%A9nez',
 'Sanford Bishop Jr.': 'https://en.wikipedia.org/wiki/Sanford_Bishop',
 'Michael K. Simpson': 'https://en.wikipedia.org/wiki/Mike_Simpson',
 'Jesus Garcia': 'https://en.wikipedia.org/wiki/Chuy_Garc%C3%ADa',
 'Danny K. Davis': 'https://en.wikipedia.org/wiki/Danny_Davis_(Illinois_politician)',
 'Frank Mrvan': 'https://en.wikipedia.org/wiki/Frank_J._Mrvan',
 'Marlin A. Stutzman': 'https://en.wikipedia.org/wiki/Marlin_Stutzman',
 'André Carson': 'https://en.wikipedia.org/wiki/Andr%C3%A9_Carson',
 'John N. Hostettler': 'https://en.wikipedia.org/wiki/John_Hostettler',
 'James Comer Jr.': 'https://en.wikipedia.org/wiki/James_Comer_(politician)',
 'Troy Carter': 'https://en.wikipedia.org/wiki/Troy_Carter_(politician)',
 'Clay Higgins': 'https://en.wikipedia.org/wiki/Clay_Higgins',
 'Mike Johnson': 'https://en.wikipedia.org/wiki/Mike_Johnson_(Louisiana_politician)',
 'Michael Soboleski': 'https://en.wikipedia.org/wiki/Mike_Soboleski',
 'Andrew Harris': 'https://en.wikipedia.org/wiki/Andy_Harris_(politician)',
 'Curtis Hertel': 'https://en.wikipedia.org/wiki/Curtis_Hertel_Jr.',
 'Matt Collier': 'https://en.wikipedia.org/wiki/Matthew_S._Collier',
 'Mary Waters': 'https://en.wikipedia.org/wiki/Mary_D._Waters',
 'Betty McCollum': 'https://en.wikipedia.org/wiki/Betty_McCollum',
 'Ilhan Omar': 'https://en.wikipedia.org/wiki/Ilhan_Omar',
 'Tom Emmer': 'https://en.wikipedia.org/wiki/Tom_Emmer',
 'Ryan K. Zinke': 'https://en.wikipedia.org/wiki/Ryan_Zinke',
 'Ed Case': 'https://en.wikipedia.org/wiki/Ed_Case',                   
 'Adam Hollier': 'https://en.wikipedia.org/wiki/Adam_Hollier',
 'Adam Morgan': 'https://en.wikipedia.org/wiki/Adam_Morgan_(politician)',
 'Anthony Sabatini': 'https://en.wikipedia.org/wiki/Anthony_Sabatini',
 'Atif Qarni': 'https://en.wikipedia.org/wiki/Atif_Qarni',
 'Becky Whitley': 'https://en.wikipedia.org/wiki/Becky_Whitley',
 'Brad Kohler': 'https://en.wikipedia.org/wiki/Brad_Kohler',
 'Briana Sewell': 'https://en.wikipedia.org/wiki/Briana_Sewell',
 'Cara Mund': 'https://en.wikipedia.org/wiki/Cara_Mund',
 'Catherine Templeton': 'https://en.wikipedia.org/wiki/Catherine_Templeton',
 'Dan Helmer': 'https://en.wikipedia.org/wiki/Dan_Helmer',
 'Dan Schwartz': 'https://en.wikipedia.org/wiki/Dan_Schwartz',
 'Denny Rehberg': 'https://en.wikipedia.org/wiki/Denny_Rehberg',
 'Eileen Filler-Corn': 'https://en.wikipedia.org/wiki/Eileen_Filler-Corn',
 'Elizabeth Helgelien': 'https://en.wikipedia.org/wiki/Elizabeth_Helgelien',
 'Elsie Arntzen': 'https://en.wikipedia.org/wiki/Elsie_Arntzen',
 'Garret Graves': 'https://en.wikipedia.org/wiki/Garret_Graves',
 'Jamie McLeod-Skinner': 'https://en.wikipedia.org/wiki/Jamie_McLeod-Skinner',
 'Jennifer Boysko': 'https://en.wikipedia.org/wiki/Jennifer_Boysko',
 'Kenneth Bogner': 'https://en.wikipedia.org/wiki/Kenneth_Bogner',
 'Krystle Kaul': 'https://en.wikipedia.org/wiki/Krystle_Kaul',
 'Mandisha Thomas': 'https://en.wikipedia.org/wiki/Mandisha_Thomas',
 'Michelle Maldonado': 'https://en.wikipedia.org/wiki/Michelle_Maldonado',
 'Mike Crane': 'https://en.wikipedia.org/wiki/Mike_Crane',
 'Mike Dugan': 'https://en.wikipedia.org/wiki/Mike_Dugan_(politician)',
 'Philip Singleton': 'https://en.wikipedia.org/wiki/Philip_Singleton',
 'Ravinder Bhalla': 'https://en.wikipedia.org/wiki/Ravinder_Bhalla',
 'Ric Holden': 'https://en.wikipedia.org/wiki/Ric_Holden',
 'Rick Becker': 'https://en.wikipedia.org/wiki/Rick_Becker',
 'Roland Riemers': 'https://en.wikipedia.org/wiki/Roland_Riemers',
 'Steve Held': 'https://en.wikipedia.org/wiki/Steve_Held',
 'Stewart Jones': 'https://en.wikipedia.org/wiki/Stewart_Jones',
 'Susheela Jayapal': 'https://en.wikipedia.org/wiki/Susheela_Jayapal',
 'Timothy M. Kennedy': 'https://en.wikipedia.org/wiki/Tim_Kennedy_(politician)'
}

for k,v in missing_house_wikis.items():
    i = dfh.index[dfh['candidate'] == k][0]
    
    val_wiki = v.split("/", 4)[4].replace("_", " ")

    dfh.loc[i, 'Wiki page (validated)'] = val_wiki
    

In [11]:
len(dfh.loc[(dfh['Wiki page (validated)'].notnull())])

515

In [12]:
# Manually fixing entries for wiki page validated. Need to look into how to avoid this mistake
dfh.loc[(dfh['candidate']=='Mike Rogers') & (dfh['State']=='MD'), 'Wiki page (validated)'] = 'Mike Rogers (Maryland_politician)'
dfh.loc[(dfh['candidate']=='Richard Grayson') & (dfh['State']=='ID'), 'Wiki page (validated)'] = None

### Address candidate's listed multiple times (same names)

In [13]:
# There are some candidates listed multiple times
grouped_cand = dfh.groupby('candidate')

# Filter groups with more than one row
filtered_groups = grouped_cand.filter(lambda x: len(x) > 1)

# Update candidate names that have duplicates
dup_names = filtered_groups['candidate'].unique()
update_count = {}
def update_name(name):
    if name not in dup_names:
        return name
        
    if name in update_count:
        update_count[name] += 1
    else:
        update_count[name] = 1

    count = update_count[name]
    return f"{name} ({count})"

dfh['candidate'] = dfh['candidate'].apply(update_name)

### Save the dataframe to a pickle

In [15]:
dfh['incumbency'] = dfh['incumbency'].fillna('Challenger')
dfh.to_pickle(f'../data/US_House_2024_df_updated_{date}.pickle')